In [31]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
from collections import Counter
import re
# pip install tqdm
from tqdm import tqdm 
import numpy as np
import pandas as pd
import itertools
from scipy.sparse import csr_matrix

In [1]:
! ls /tmp/working/machinleaning/ml/6/ident/kaggle_data/train/30/

user0048.csv  user0120.csv  user0361.csv  user0512.csv	user0884.csv
user0049.csv  user0149.csv  user0370.csv  user0543.csv	user0912.csv
user0050.csv  user0251.csv  user0405.csv  user0575.csv	user0917.csv
user0059.csv  user0318.csv  user0445.csv  user0660.csv	user0939.csv
user0068.csv  user0324.csv  user0454.csv  user0762.csv	user0984.csv
user0104.csv  user0352.csv  user0468.csv  user0812.csv	user0989.csv


In [7]:
PATH_TO_DATA = '/tmp/working/machinleaning/ml/6/ident/kaggle_data/'

In [32]:
def get_list(user_data,site_freq,\
             session_length=10, window_size=10):
         
        N=user_data.shape[0]
        r_b = lambda x: x+session_length
        slice_list=((i,r_b(i) if r_b(i)<N else N) for i in range(0,N,window_size))
        
        list_sites = []
        
        for pair in slice_list:
            lst = list(\
                map(lambda x: site_freq[x],user_data.site.values[pair[0]:pair[1]]))
            list_sites.append(lst)
        return list_sites
    
def prepare_sparse_train_set_window(path_to_csv_files, site_freq_path, 
                                    session_length=10, window_size=10):
    
    ''' ВАШ КОД ЗДЕСЬ'''
    pat = re.compile("user([\d]+)[.]")
    indptr = [0]
    count = 0
    indices = []
    data = []
    y = []
    
#     from IPython.core.debugger import Tracer; Tracer()()
    
    with open(site_freq_path,"rb") as f:
        site_freq = pickle.load(f)

    for f in sorted(glob(path_to_csv_files+'/*')):
        user_data = pd.read_csv(f)
        lst_ = get_list(user_data,site_freq,session_length,window_size)
        for col in lst_:
            counter = Counter(col)
            count += len(counter)
            indptr.append(count)
            indices += counter.keys()
            data += counter.values()
        y.append([int(re.search(pat,f).group(1))]*len(lst_))
    y=list(itertools.chain(*y))
    return \
    csr_matrix((data, indices, indptr),\
               shape=(len(y),len(site_freq)))[:,1:],np.array(y,dtype=np.int32)
    # X = csr_matrix((data, indices, indptr), shape=(X.shape[0], n))[:,1:]

In [34]:
X_30_10_10, y_30_10_10=prepare_sparse_train_set_window(os.path.join(PATH_TO_DATA,'train/30/'),\
                                                os.path.join(PATH_TO_DATA,'site_dic.pkl'))

In [37]:
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_30users.pkl'), 'wb') as X30_pkl:
    pickle.dump(X_30_10_10, X30_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_30users.pkl'), 'wb') as y30_pkl:
    pickle.dump(y_30_10_10, y30_pkl, protocol=2)

In [41]:
num_users = 30
for window_size, session_length in tqdm(itertools.product([10, 7, 5], [15, 10, 7, 5])):
    if window_size <= session_length and (window_size, session_length) != (10, 10):
        X_sparse, y = prepare_sparse_train_set_window(os.path.join(PATH_TO_DATA,str(num_users)+'users'), 
                                                   os.path.join(PATH_TO_DATA, 
                       'y_30users.pkl'.format(num_users)),
                                   session_length, window_size)
        with open(os.path.join(PATH_TO_DATA, 
                   'X_sparse_{0}users_s{1}_w{2}.pkl'.\
                               format(num_users,session_length,window_size)), 'wb') as X_pkl:
            pickle.dump(X_sparse, X_pkl, protocol=2)

        with open(os.path.join(PATH_TO_DATA, 
                   'y_{0}users_s{1}_w{2}.pkl'.\
                               format(num_users,session_length,window_size)), 'wb') as y_pkl:
            pickle.dump(y, y_pkl, protocol=2)

12it [00:00, 1138.73it/s]
